GAN Strcuture (GRU)


In [1]:
%pip install torch==1.8.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
%pip install torchtext==0.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 645.4 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0+cu111 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.8.0+cu111 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0+cu111 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━

In [94]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import csv
import numpy as np
import random
import torch.optim as optim
import torch.utils.data
import torchtext
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import json
import ast
import glob
import re
import string
import collections
from torchtext.legacy import data

class Discriminator(nn.Module):

    def __init__(self,vocab_size,hidden_size,n_layers=1):
        super(Discriminator, self).__init__()
        self.emb = torch.eye(vocab_size)
        self.hidden_size = hidden_size
        self.rnn = nn.GRU(vocab_size,hidden_size,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,50)
        self.fc2 = nn.Linear(50,1)

    def forward(self, x, hidden = None):
        
        x = self.emb[x]
        out, hidden = self.rnn(x,hidden)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return torch.sigmoid(out)

class Generator(nn.Module):

    def __init__(self,vocab_size,hidden_size,n_layers=1):
        super(Generator, self).__init__()
        self.emb = torch.eye(vocab_size)
        self.rnn = nn.GRU(vocab_size,hidden_size, n_layers,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,vocab_size)

    def forward(self, x, hidden = None):
        x = self.emb[x]
        out, hidden = self.rnn(x,hidden)
        out = self.fc1(out)
        return out



Previous Training (Example From Tutorial)


In [87]:
def train (trainDS,G,D,lr=0.002,batch_size=1,num_epochs=10):
  d_optimizer = optim.Adam(D.parameters(), lr)
  g_optimizer = optim.Adam(G.parameters(), lr)

  criterion = nn.MSELoss()

  train_data = trainDS
  #train_loader = train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

  samples = []
  losses = []

  for epoch in range(num_epochs):
    D.train()
    G.train()

    for batch_i in range(len(train_data)):
   # for batch_i,  real_recipes,ingredients) in enumerate(train_data['directions'],train_data['ingredients']):
            real_recipes = train_data['directions'][batch_i]
            ingredients = train_data['ingredients'][batch_i]

            # batch_size = real_recipes.size(0)

            # === Train the Discriminator ===
            
            d_optimizer.zero_grad()

            # discriminator losses on real images 
          
            D_real = D(real_recipes)
            labels = torch.ones(batch_size)

           
            D_real= sum(D_real)/D_real.shape[0]
            d_real_loss = criterion(D_real, labels)

            
            # discriminator losses on fake images
            # z = np.random.uniform(-1, 1, size=(batch_size, rand_size))
            # z = torch.from_numpy(z).float()
            
            ingredients = torch.Tensor(ingredients) 
            ingredients = ingredients.to(torch.long)

            fake_recipes = G(ingredients)

            fake_recipes = torch.Tensor(fake_recipes) 
            fake_recipes = fake_recipes.to(torch.long)
            D_fake = D(fake_recipes)
            
            #labels = torch.zeros(batch_size) # fake labels = 0
            labelsD = torch.zeros(1)
            labelsD = torch.diag(labelsD,0)
            d_fake_loss = criterion(D_fake, labelsD)
            
            # add up losses and update parameters
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()


             # === Train the Generator ===
            g_optimizer.zero_grad()
            
            # generator losses on fake images
            # z = np.random.uniform(-1, 1, size=(batch_size, rand_size))
            # z = torch.from_numpy(z).float()

            ingredients = torch.Tensor(ingredients) 
            ingredients = ingredients.to(torch.long)

            fake_recipes = G(ingredients)

            fake_recipes = torch.Tensor(fake_recipes) 
            fake_recipes = fake_recipes.to(torch.long)

            D_fake = D(fake_recipes)
            #labels = torch.ones(batch_size) #flipped labels


            labels = torch.ones(batch_size)
            # compute loss and update parameters
            g_loss = criterion(D_fake,labels)
            g_loss.backward()
            g_optimizer.step()


            print('Epoch [%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
                % (epoch + 1, num_epochs, d_loss.item(), g_loss.item()))

            # append discriminator loss and generator loss
            losses.append((d_loss.item(), g_loss.item()))


    return losses


# New Training

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Setting HyperParameters

In [33]:
lr = 3e-4
batch_size = 12
num_epochs = 2


Formatting Dataset (From Baseline)

In [88]:
patterns = [r'\'', r'\"', r'\.', r'<br \/>', r',', r'\(', r'\)', r'\!', r'\?', r'\:', r'\s+']
replacements = [' \'  ', '', ' . ', ' ', ' , ', ' ( ', ' ) ', ' ! ', ' ? ', ' ', ' ']
patterns_dict = list((re.compile(p), r) for p, r in zip(patterns, replacements))

def basic_english_normalize(line):
    line = line.lower()
    for pattern_re, replaced_str in patterns_dict:
        line = pattern_re.sub(replaced_str, line)
    return line.split()

directions_field = torchtext.legacy.data.Field(sequential=True,
                                  tokenize=basic_english_normalize,
                                  include_lengths=True,
                                  batch_first=False,
                                  use_vocab=True,
                                  init_token="<BOS>",
                                  eos_token="<EOS>")

fields = [('directions', directions_field), ('ingredients', None)]
baseline_data = torchtext.legacy.data.TabularDataset("drive/MyDrive/data/char_nlg_df.csv", "csv", fields)

directions_field.build_vocab(baseline_data)
directions_field.vocab = torchtext.vocab.Vocab(directions_field.vocab.freqs, specials=['<unk>','<pad>', '<BOS>', '<EOS>', ';'])
baseline_vocab_stoi = directions_field.vocab.stoi
baseline_vocab_itos = directions_field.vocab.itos
baseline_vocab_size = len(directions_field.vocab.itos)
print(baseline_vocab_size)
print(directions_field.vocab.itos)

3502
['<unk>', '<pad>', '<BOS>', '<EOS>', ';', 'and', 'in', 'to', 'add', 'with', 'minutes', 'until', 'a', 'mix', 'for', '1', 'the', 'bake', 'at', 'of', 'into', '2', 'on', 'ingredients', 'sugar', 'over', 'or', 'pour', 'stir', 'pan', 'mixture', '350', 'well', 'cook', 'butter', 'cheese', 'heat', 'water', 'together', 'top', 'inch', 'combine', 'salt', 'flour', 'cream', 'all', 'place', 'oven', 'bowl', 'brown', 'cool', 'milk', 'beat', 'baking', 'x', '4', '3', 'chicken', 'eggs', 'cover', 'put', 'onion', 'about', 'dish', 'sprinkle', 'pepper', 'then', 'cup', 'large', '9', 'serve', '10', 'cut', 'greased', 'drain', '30', 'remaining', 'is', 'remove', 'hours', 'boil', 'from', 'sauce', 'let', 'spread', 'hour', 'egg', 'vanilla', '8', '5', 'cake', 'oil', 'makes', 'set', '6', 'hot', '13', '20', 'nuts', '15', 'meat', 'margarine', 'dry', 'simmer', 'roll', 'stirring', 'casserole', 'medium', 'chill', 'each', 'blend', 'soup', 'juice', 'tender', 'pie', 'melt', 'layer', 'potatoes', 'skillet', 'sheet', 'if', 's

In [89]:
def sample_sequence(model, max_len=1000, temperature=0.8):
    generated_sequence = ""
    
    inp = torch.Tensor([baseline_vocab_stoi["<BOS>"]]).long()
    hidden = None
    step = 1

    for c in range(max_len):
          output, hidden = model(inp.unsqueeze(0), hidden)
          output_dist = output.data.view(-1).div(temperature).exp()
          top = int(torch.multinomial(output_dist, 1)[0])

          predicted_char = baseline_vocab_itos[top]

          if predicted_char == "<pad>":
              continue

          if predicted_char == "<BOS>":
              continue
          
          if predicted_char == "<unk>":
              continue

          if predicted_char == ";":
              step += 1
              predicted_char = str("\n " + str(step) + ".")

          if predicted_char == "<EOS>":
              break

          generated_sequence += predicted_char + " "
          inp = torch.Tensor([top]).long()

    return generated_sequence

In [90]:
def train_GAN(descriminator, generator, data, lr, batch_size, num_epochs):
  d_optimizer = optim.Adam(descriminator.parameters(), lr)
  g_optimizer = optim.Adam(generator.parameters(), lr)

  criterion = nn.MSELoss()
  
  train_data = data
  #train_loader = train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
  data_iter = torchtext.legacy.data.BucketIterator(data, batch_size=batch_size, sort_key=lambda x: len(x.directions), sort_within_batch=True)

  samples = []
  losses = []

  for epoch in range(num_epochs):
    descriminator.train()
    generator.train()

    for (directions, lengths), ingredients in data_iter:
   # for batch_i,  real_recipes,ingredients) in enumerate(train_data['directions'],train_data['ingredients']):
            real_directions = directions
            #print(real_directions)

            # batch_size = real_recipes.size(0)

            # === Train the Discriminator ===
            
            d_optimizer.zero_grad()

            # discriminator losses on real recipe 
          
            D_real = descriminator(real_directions)
            labels = torch.ones(batch_size)

           
            D_real= sum(D_real)/D_real.shape[0]
            d_real_loss = criterion(D_real, labels)

            
            # discriminator losses on fake recipe
            # z = np.random.uniform(-1, 1, size=(batch_size, rand_size))
            # z = torch.from_numpy(z).float()
            
            ingredients = torch.Tensor(ingredients) 
            ingredients = ingredients.to(torch.long)

            fake_recipes = generator(ingredients)

            fake_recipes = torch.Tensor(fake_recipes) 
            fake_recipes = fake_recipes.to(torch.long)
            D_fake = descriminator(fake_recipes)
            
            #labels = torch.zeros(batch_size) # fake labels = 0
            labelsD = torch.zeros(1)
            labelsD = torch.diag(labelsD,0)
            d_fake_loss = criterion(D_fake, labelsD)
            
            # add up losses and update parameters
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()


             # === Train the Generator ===
            g_optimizer.zero_grad()
            
            # generator losses on fake images
            # z = np.random.uniform(-1, 1, size=(batch_size, rand_size))
            # z = torch.from_numpy(z).float()

            ingredients = torch.Tensor(ingredients) 
            ingredients = ingredients.to(torch.long)

            fake_recipes = generator(ingredients)

            fake_recipes = torch.Tensor(fake_recipes) 
            fake_recipes = fake_recipes.to(torch.long)

            D_fake = descriminator(fake_recipes)
            #labels = torch.ones(batch_size) #flipped labels


            labels = torch.ones(batch_size)
            # compute loss and update parameters
            g_loss = criterion(D_fake,labels)
            g_loss.backward()
            g_optimizer.step()


            print('Epoch [%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
                % (epoch + 1, num_epochs, d_loss.item(), g_loss.item()))

            # append discriminator loss and generator loss
            losses.append((d_loss.item(), g_loss.item()))


    return losses

In [93]:
disc = Discriminator(baseline_vocab_size, 64, n_layers=1)
gen = Generator(baseline_vocab_size, 64, n_layers=1)
train_GAN(disc, gen, baseline_data, lr=lr, batch_size=batch_size, num_epochs=num_epochs)

tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
        [ 236,   49,   80,   13,  164,   13,   49,   13,   41,  594,  318,   28],
        [ 127,  379,   57,   45,  116,   45,   45,   45,   45,  758,    5,    5],
        [ 154,    5,    5,   23,    4,   23,  100,   23,   23,  782,   17,  234],
        [   8,  126,  961,   20,   13,    4,   38,    4,   13,    5,  497,  164],
        [ 276,   74,    4,   12,  378,    6,    9,   27,   32,  110,  267,   56],
        [1013,    4,   33,    4,  346,   68,  126,   20,    4,   45,    4,   23],
        [   5,    8,  228,   48,  143,  529,    4,   73,   59,   23,   50,   14],
        [  21,  187,  266,    4,    9,  241,   74,   56,    5,   38,    4,   56],
        [ 714,  473,    7,   27,  124,    5,    4,  173,  123,  367,  438,   10],
        [ 628,  136,  256,  209,  297,   33,    8,  106,   88,   11,  677,    4],
        [ 304,  134,  273,  225,    5,   22,   35,   17,   79,  274,    4,   28],
        [ 175,  

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: ignored